In [3]:
# Handful of configuration

import os
import boto3
import json
from sagemaker import get_execution_role, session

region= boto3.Session().region_name

sm_client = boto3.client('sagemaker')

role = get_execution_role()
print("RoleArn: {}".format(role))

RoleArn: arn:aws:iam::376678947624:role/vegetation-management-works-SageMakerExecutionRole-OZ2K30BYST0I


In [2]:
%store -r endpoint_name
%store -r default_bucket
%store -r current_endpoint_capture_prefix
%store -r s3_key_prefix

In [7]:
!aws s3 sync s3://{default_bucket}/{current_endpoint_capture_prefix} workspace/data

download: s3://sagemaker-us-west-2-376678947624/Mikael110-llama-2-7b-guanaco-fp16/datacapture/llama-2-7b-2023-10-21-02-26-02-152-endpoint/AllTraffic/2023/10/21/02/35-09-353-1f41490f-4e1e-4c43-8d9a-2bb25433a0e6.jsonl to workspace/data/AllTraffic/2023/10/21/02/35-09-353-1f41490f-4e1e-4c43-8d9a-2bb25433a0e6.jsonl
download: s3://sagemaker-us-west-2-376678947624/Mikael110-llama-2-7b-guanaco-fp16/datacapture/llama-2-7b-2023-10-21-02-26-02-152-endpoint/AllTraffic/2023/10/21/03/39-07-208-5459146d-0729-4374-b559-5b391308ce08.jsonl to workspace/data/AllTraffic/2023/10/21/03/39-07-208-5459146d-0729-4374-b559-5b391308ce08.jsonl
download: s3://sagemaker-us-west-2-376678947624/Mikael110-llama-2-7b-guanaco-fp16/datacapture/llama-2-7b-2023-10-21-02-26-02-152-endpoint/AllTraffic/2023/10/21/03/41-03-334-9a247cda-7335-4bb7-b896-05f56d5b1afd.jsonl to workspace/data/AllTraffic/2023/10/21/03/41-03-334-9a247cda-7335-4bb7-b896-05f56d5b1afd.jsonl


## Test script locally

In [19]:
os.environ['dataset_source'] = f'{os.getcwd()}/workspace/data'
os.environ['output_path'] = f'{os.getcwd()}/workspace/output'

!python workspace/src/llm_monitoring.py

['Tell me a fun fact about Boca Raton, Florida', 'What is an anemone?', 'What are some quick ways to lose all of my money?', 'What is core banking?', 'What are some items that you might see in a fridge?', 'what can we do when coffee spill on laptop to make it working', 'Using examples taken from the paragraph, provide the major risks to humans with climate change in a short bulleted list', 'How many world championships has Max Verstappen won?', 'Which is a species of fish? Tetra or Quart', 'Why is pricing important in the overall strategy of a product?']
['Boca Raton is known for various items including:', 'An anemone is a flower with multiple petals that are joined at the center, forming a shape that resembles a cup. The petals are usually arranged in a radial pattern, with the center of the flower being the hub of the anemone. The flower is typically pink, red, or white in color, and is characterized by its delicate and intricate appearance. The anemone is a very popular flower choic

## Test container locally

In [27]:
!pygmentize workspace/Dockerfile

FROM python:3.9-slim-buster

RUN pip3 install botocore boto3==1.28.67 langchain==0.0.319

WORKDIR /home

COPY src/* /home/
COPY data/ /home/data

ENTRYPOINT ["python3", "llm_monitoring.py"]


In [34]:
!cd workspace && docker build -t workspace .

Sending build context to Docker daemon  60.93kB
Step 1/5 : FROM python:3.9-slim-buster
 ---> c84dbfe3b8de
Step 2/5 : RUN pip3 install botocore boto3==1.28.67 langchain==0.0.319
 ---> Using cache
 ---> b4962bb1a66f
Step 3/5 : WORKDIR /home
 ---> Using cache
 ---> 6099b9f6c3ab
Step 4/5 : COPY src/* /home/
 ---> Using cache
 ---> b88bcd529691
Step 5/5 : ENTRYPOINT ["python3", "llm_monitoring.py"]
 ---> Running in ec21de85cc29
Removing intermediate container ec21de85cc29
 ---> cde8e3c17dd9
Successfully built cde8e3c17dd9
Successfully tagged workspace:latest


In [39]:
!docker run -v {os.getcwd()}/workspace/data:/home/data -v {os.getcwd()}/workspace/output:/home/output -e dataset_source=data/ -e output_path=output workspace

['Tell me a fun fact about Boca Raton, Florida', 'What is an anemone?', 'What are some quick ways to lose all of my money?', 'What is core banking?', 'What are some items that you might see in a fridge?', 'what can we do when coffee spill on laptop to make it working', 'Using examples taken from the paragraph, provide the major risks to humans with climate change in a short bulleted list', 'How many world championships has Max Verstappen won?', 'Which is a species of fish? Tetra or Quart', 'Why is pricing important in the overall strategy of a product?']
['Boca Raton is known for various items including:', 'An anemone is a flower with multiple petals that are joined at the center, forming a shape that resembles a cup. The petals are usually arranged in a radial pattern, with the center of the flower being the hub of the anemone. The flower is typically pink, red, or white in color, and is characterized by its delicate and intricate appearance. The anemone is a very popular flower choic

## Bring your own custom model drift detection algorithm

In order to bring your own custom model drift detection algorithm, you need to do following things:
* Create custom detection algorithms. We have included algorithms under src folder
* Create a Docker container.
* Set enviornmental variables where the container can find the datacapture data from SageMaker Model Monitor. These variables have to match with the values we provide to monitor scheduler later.

In [1]:
from docker_utils import build_and_push_docker_image

repository_short_name = 'custom-llm-monitor'

image_name = build_and_push_docker_image(repository_short_name, dockerfile='workspace/Dockerfile', context='workspace')

Building docker image custom-llm-monitor from workspace/Dockerfile
$ docker build -t custom-llm-monitor -f workspace/Dockerfile workspace
Sending build context to Docker daemon  61.44kB
Step 1/5 : FROM python:3.9-slim-buster
 ---> c84dbfe3b8de
Step 2/5 : RUN pip3 install botocore boto3==1.28.67 langchain==0.0.319
 ---> Using cache
 ---> b4962bb1a66f
Step 3/5 : WORKDIR /home
 ---> Using cache
 ---> 6099b9f6c3ab
Step 4/5 : COPY src/* /home/
 ---> Using cache
 ---> b88bcd529691
Step 5/5 : ENTRYPOINT ["python3", "llm_monitoring.py"]
 ---> Using cache
 ---> cde8e3c17dd9
Successfully built cde8e3c17dd9
Successfully tagged custom-llm-monitor:latest
Done building docker image custom-llm-monitor
Created new ECR repository: custom-llm-monitor
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/refere

In [6]:
s3_result_path = f's3://{default_bucket}/{s3_key_prefix}/result/{endpoint_name}'

sm_client.create_monitoring_schedule(
    MonitoringScheduleName=endpoint_name,
    MonitoringScheduleConfig={
        'ScheduleConfig': {
            'ScheduleExpression': 'cron(0 * ? * * *)'
        },
        'MonitoringJobDefinition': {
            'MonitoringInputs': [
                {
                    'EndpointInput': {
                        'EndpointName': endpoint_name,
                        'LocalPath': '/opt/ml/processing/endpointdata'
                    }
                },
            ],
            'MonitoringOutputConfig': {
                'MonitoringOutputs': [
                    {
                        'S3Output': {
                            'S3Uri': s3_result_path,
                            'LocalPath': '/opt/ml/processing/resultdata',
                            'S3UploadMode': 'EndOfJob'
                        }
                    },
                ]
            },
            'MonitoringResources': {
                'ClusterConfig': {
                    'InstanceCount': 1,
                    'InstanceType': 'ml.c5.xlarge',
                    'VolumeSizeInGB': 10
                }
            },
            'MonitoringAppSpecification': {
                'ImageUri': image_name,
            },
            'StoppingCondition': {
                'MaxRuntimeInSeconds': 600
            },
            'Environment': {
                'string': 'string'
            },
            'RoleArn': role
        }
    }
)

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-west-2:376678947624:monitoring-schedule/llama-2-7b-2023-10-21-02-26-02-152-endpoint',
 'ResponseMetadata': {'RequestId': '398773ac-60fd-48b3-8d30-427eb28b7a79',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '398773ac-60fd-48b3-8d30-427eb28b7a79',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '132',
   'date': 'Sat, 21 Oct 2023 22:43:16 GMT'},
  'RetryAttempts': 0}}

## Run monitoring job immediately

In [9]:
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput
from urllib.parse import urlparse

# region
# role
data_capture_path=f's3://{default_bucket}/{current_endpoint_capture_prefix}'
# s3_result_path
instance_count=1
instance_type='ml.c5.xlarge'
# publish_cloudwatch_metrics='Disabled'

data_capture_sub_path = data_capture_path[data_capture_path.rfind('datacapture/') :]
data_capture_sub_path = data_capture_sub_path[data_capture_sub_path.find('/') + 1 :]

input_1 = ProcessingInput(input_name='input_1',
                      source=data_capture_path,
                      destination='/opt/ml/processing/input/endpoint/' + data_capture_sub_path,
                      s3_data_type='S3Prefix',
                      s3_input_mode='File')

outputs = ProcessingOutput(output_name='result',
                           source='/opt/ml/processing/output',
                           destination=s3_result_path,
                           s3_upload_mode='Continuous')

env = {'dataset_source': '/opt/ml/processing/input/endpoint',
       'output_path': '/opt/ml/processing/output'}

processor = Processor(image_uri = image_name,
                      instance_count = instance_count,
                      instance_type = instance_type,
                      role=role,
                      env = env)

processor.run(inputs=[input_1], outputs=[outputs])

INFO:sagemaker:Creating processing-job with name custom-llm-monitor-2023-10-21-22-57-33-667


s3://sagemaker-us-west-2-376678947624/Mikael110-llama-2-7b-guanaco-fp16/datacapture/llama-2-7b-2023-10-21-02-26-02-152-endpoint
llama-2-7b-2023-10-21-02-26-02-152-endpoint
.......................['Why some people are more stressed than others and how to manage stress?', 'Which episode of The X-Files did Dana Scully get diagnosed with cancer?', 'Why do I have a belly button?', 'Tell me a fun fact about Boca Raton, Florida', 'What is an anemone?', 'What are some quick ways to lose all of my money?', 'What is core banking?', 'What are some items that you might see in a fridge?', 'what can we do when coffee spill on laptop to make it working', 'Using examples taken from the paragraph, provide the major risks to humans with climate change in a short bulleted list', 'How many world championships has Max Verstappen won?', 'Which is a species of fish? Tetra or Quart', 'Why is pricing important in the overall strategy of a product?']
["First, let's understand stress and its causes.", 'Season fi